In [1]:
import os, sys
sys.path.append(os.path.dirname(os.getcwd()) + '/source')

import numpy as np
import matplotlib.pyplot as plt
import SolveLinSys
import time
from utilities import compute_derivatives_2d, find_nearest_value
from numba import njit

In [2]:
def false_transient_one_iteration(stateSpace, A, B_z, B_y, C_zz, C_yy, D, v0, ε=.3, tol=-10):
    A = A.reshape(-1, 1, order='F')
    B = np.hstack([B_z.reshape(-1, 1, order='F'), B_y.reshape(-1, 1, order='F')])
    C = np.hstack([C_zz.reshape(-1, 1, order='F'), C_yy.reshape(-1, 1, order='F')])
    D = D.reshape(-1, 1, order='F')
    v0 = v0.reshape(-1, 1, order='F')
    out = SolveLinSys.solveFT(stateSpace, A, B, C, D, v0, ε, tol)
    return out

Equation:

\begin{align}
0 = \max_e \min_{h_2} \min_{\pi_j:\sum_j^J{\pi_j}=1} &-\delta \phi(y,z_2) + \delta\eta\log e + \xi_m \frac{(h_2)^2}{2} + \frac{\partial \phi}{\partial y}(y, z_2) e\\
&+ \left[\frac{\partial \phi}{\partial z_2}(y, z_2)\right]\left[-\rho(z_2 - \mu_2) + \sqrt{z_2}\sigma_2 h_2\right] + \left[\frac{\partial^2 \phi}{\partial (z_2)^2}(y, z_2)\right]\left(\frac{z_2|\sigma_2|^2}{2}\right)\\
& + (\eta-1)\sum_{j}^J \pi_{j}\left(\frac{d}{d \tau}\Lambda_j(z_2 y)\left(z_2 e-y\rho(z_2-\mu_2)+ y\sqrt{z_2}\sigma_2h_2\right) + \frac{1}{2}\frac{d^2}{d\tau^2}\Lambda_j(z_2y)z_2y^2|\sigma_2|^2\right)\\
& + \xi_a \sum_{j}^J \pi_{j} \left(\log \pi_j - \log \pi_j^o\right)
\end{align}

where

\begin{equation}
\Lambda_j(\tau)=
\begin{cases}
  \lambda_1 \tau + \frac{\lambda_2}{2}\tau^2, & \tau\leq \bar{\tau} \\
  \lambda_1 \tau + \frac{\lambda_2}{2}\tau^2 + \frac{\lambda_{2,j}^+}{2}(\tau-\bar{\tau})^2, & \tau > \bar{\tau}
\end{cases}
\end{equation}

FOC of $\pi_j$ gives:
$$
\pi_j \propto \exp\left[-\frac{1}{\xi_a}(\eta-1)\left(\frac{d}{d \tau}\Lambda_j(z_2 y)\left(z_2 e-y\rho(z_2-\mu_2)+ y\sqrt{z_2}\sigma_2h_2\right) + \frac{1}{2}\frac{d^2}{d\tau^2}\Lambda_j(z_2y)z_2y^2|\sigma_2|^2\right)\right]\pi_j^o
$$

FOC of $h_2$ gives : 

\begin{align}
h_2^* = -\frac{\frac{\partial \phi}{\partial z_2}(y, z_2)\sqrt{z_2}\sigma_2 + (\eta-1)\sum_{j}^J \pi_j \frac{d}{d \tau}\Lambda_j(z_2 y)\left(y\sqrt{z_2}\sigma_2\right)}{\xi_m}
\end{align}

FOC of $e$ gives :

$$
e^* = -\frac{\delta \eta}{\frac{\partial \phi}{\partial y}(y, z_2) + (\eta-1)\sum_{j}^J\pi_j\frac{d}{d \tau}\Lambda_j(z_2 y)z_2}
$$

In [3]:
def false_transient(z_grid, y_grid, model_paras=(), v0=None, ϵ=.5, tol=1e-8, max_iter=10_000):
    η, ξ_m, ξ_a, δ, η, μ_2, ρ, σ_2, λ_1, λ_2, λ_bar, λ_2p, π_o  = model_paras
    Δ_z = z_grid[1] - z_grid[0]
    Δ_y = y_grid[1] - y_grid[0]
    (z_mat, y_mat) = np.meshgrid(z_grid, y_grid, indexing = 'ij')
    stateSpace = np.hstack([z_mat.reshape(-1, 1, order='F'), y_mat.reshape(-1, 1, order='F')])
    if v0 is None:
        v0 = -δ*η*y_mat

    d_Λ = np.array([λ_1 + λ_2*y_mat*z_mat + 
                            λ_2p_j*(y_mat*z_mat>λ_bar)*(y_mat*z_mat-λ_bar) for λ_2p_j in λ_2p])
    dd_Λ = np.array([λ_2 + λ_2p_j*(y_mat*z_mat>λ_bar) for λ_2p_j in λ_2p])
    d_Λ_yz_sigma = d_Λ*y_mat*np.sqrt(z_mat)*σ_2
    d_Λ_z = d_Λ*z_mat
    mean_reversion = - y_mat*ρ*(z_mat-μ_2)
    std = np.sqrt(z_mat)*σ_2
    std_y = y_mat*std
    variance = z_mat*y_mat**2*σ_2**2

    π = np.ones_like(d_Λ)
    for i in range(π.shape[0]):
        π[i] = π_o[i]
    π_o = π.copy()
    h = - (η-1.) / ξ_m * np.sum(π*d_Λ_yz_sigma, axis=0)
    e = - δ*η / ((η-1)*np.sum(π*d_Λ_z, axis=0))
    
    count = 1
    error = 1.

    while error > tol and count < max_iter:
        v_old = v0.copy()
        v0_dz = compute_derivatives_2d(v0, 0, 1, Δ_z)
        v0_dzz = compute_derivatives_2d(v0, 0, 2, Δ_z)
        v0_dy = compute_derivatives_2d(v0, 1, 1, Δ_y)

        temp = d_Λ*(z_mat*e + mean_reversion + std_y*h) + 0.5*dd_Λ*variance
        log_π_ratio = -(η-1.)*temp/ξ_a
        π_ratio = log_π_ratio - np.max(log_π_ratio, axis=0)
        π = np.exp(π_ratio) * π_o
        π = π/np.sum(π, axis=0)
        π[π<=0] = 1e-12
        
        h = - (v0_dz*std + (η-1)*np.sum(π*d_Λ_yz_sigma, axis=0)) / ξ_m
        e = - δ*η / (v0_dy + (η-1)*np.sum(π*d_Λ_z, axis=0))
        e[e<=0] = 1e-12

        A = np.ones_like(z_mat)*(-δ)
        B_z = -ρ*(z_mat - μ_2)
        B_y = e
        C_zz = z_mat*σ_2**2/2
        C_yy = np.zeros_like(z_mat)
        D =  v0_dz*np.sqrt(z_mat)*σ_2*h + δ*η*np.log(e) + ξ_m*h**2/2 + (η-1)*np.sum(π*temp, axis=0)\
            + ξ_a*np.sum(π*(np.log(π)-np.log(π_o)), axis=0)
        res = false_transient_one_iteration(stateSpace, A, B_z, B_y, C_zz, C_yy, D, v0, ϵ) # CG tol = 1e-10
        v0 = res[2].reshape(v0.shape, order = "F")

        rhs_error = A*v0 + B_z*v0_dz + B_y*v0_dy + C_zz*v0_dzz + D
        rhs_error = np.max(abs(rhs_error))
        lhs_error = np.max(abs((v0 - v_old)/ϵ))
        error = lhs_error
        print("Iteration %s: LHS Error: %s; RHS Error %s" % (count, lhs_error, rhs_error))
        count += 1
    return v0, e, h, π

In [8]:
# Model parameters
η = .032
ξ_m = .00256
ξ_a = .01
δ = .01
# Z process
μ_2 = 1.86/1000.
ρ = 0.9
σ_2 = np.sqrt(2*ρ/10416.29)/2
# Damage setting
λ_1 = 1.7675/10000
λ_2 = 0.0022*2
λ_bar = 2.

# three damage
# λ_2p = np.array([0., 0.0197*2, 0.3853*2])
# π_o = np.array([0.475, 0.475, 0.05])

# two damage
λ_2p = np.array([0., 0.0197*2])
π_o = np.array([0.5, 0.5])

pde_paras = (η, ξ_m, ξ_a, δ, η, μ_2, ρ, σ_2, λ_1, λ_2, λ_bar, λ_2p, π_o)

# Capital and z parameters
α_k = -0.043
β_k = 0.04
σ_k = np.array([0.477*2, 0])/100.
β_z = 0.014*4
σ_z = np.array([0.011*2, 0.025*2])
α = 0.115
ioverk = 0.09
κ = 6.667

# Grid setting
n_z = 50
z_min = np.max([μ_2 - 3*np.sqrt(σ_2**2*μ_2/(2*ρ)), 1e-9])
z_max = μ_2 + 3*np.sqrt(σ_2**2*μ_2/(2*ρ))
z_grid = np.linspace(z_min, z_max, n_z)

n_y = 50
y_min = 1e-10
y_max = 1500.
y_grid = np.linspace(y_min, y_max, n_y)

In [7]:
ϕ_grid, e_grid, h_grid, π_grid = false_transient(z_grid, y_grid, model_paras=pde_paras,
                                         v0=None, ϵ=.5, tol=1e-8, max_iter=10_000)

Iteration 1: LHS Error: 0.004340677580566887; RHS Error 0.006663655547487444
Iteration 2: LHS Error: 0.0031745069101982715; RHS Error 0.0038133519425316413
Iteration 3: LHS Error: 0.003629244385647601; RHS Error 0.004859703543067463
Iteration 4: LHS Error: 0.0032856416458948434; RHS Error 0.004033116159286352
Iteration 5: LHS Error: 0.003367835966971877; RHS Error 0.003956610898795662
Iteration 6: LHS Error: 0.0034317858594087314; RHS Error 0.004226930739359612
Iteration 7: LHS Error: 0.0032494065207637757; RHS Error 0.0037581923214795063
Iteration 8: LHS Error: 0.003291266862161857; RHS Error 0.0037305652329595322
Iteration 9: LHS Error: 0.0032809210473329076; RHS Error 0.0038443692510331385
Iteration 10: LHS Error: 0.00314074071939352; RHS Error 0.003483213252938674
Iteration 11: LHS Error: 0.003143664719720718; RHS Error 0.003469865573473877
Iteration 12: LHS Error: 0.003077553783026432; RHS Error 0.0034388061664900596
Iteration 13: LHS Error: 0.0029666995115936734; RHS Error 0.0031

Iteration 109: LHS Error: 0.001752876464806763; RHS Error 0.0017518770285472815
Iteration 110: LHS Error: 0.0017453044950936913; RHS Error 0.001744170492602306
Iteration 111: LHS Error: 0.0017374970382543764; RHS Error 0.001736865453101112
Iteration 112: LHS Error: 0.0017298129715674349; RHS Error 0.0017288067445472982
Iteration 113: LHS Error: 0.0017223333703521027; RHS Error 0.001721395780921173
Iteration 114: LHS Error: 0.0017145955649898736; RHS Error 0.0017137998266534829
Iteration 115: LHS Error: 0.0017070744248939285; RHS Error 0.0017060618046942128
Iteration 116: LHS Error: 0.0016996817714280965; RHS Error 0.0016988633613095492
Iteration 117: LHS Error: 0.0016920589505667327; RHS Error 0.001691078690282363
Iteration 118: LHS Error: 0.001684654467949187; RHS Error 0.0016836373048861996
Iteration 119: LHS Error: 0.0016773340602391218; RHS Error 0.0016765742403231894
Iteration 120: LHS Error: 0.0016698394890753976; RHS Error 0.0016688630765141696
Iteration 121: LHS Error: 0.001662

Iteration 211: LHS Error: 0.0011246270375765977; RHS Error 0.001123837187473802
Iteration 212: LHS Error: 0.0011198166510695273; RHS Error 0.0011190289152521078
Iteration 213: LHS Error: 0.0011150281090256953; RHS Error 0.0011142424643075903
Iteration 214: LHS Error: 0.0011102613005795625; RHS Error 0.0011094777447353964
Iteration 215: LHS Error: 0.0011055161072993647; RHS Error 0.0011047346562283382
Iteration 216: LHS Error: 0.0011007924438100614; RHS Error 0.0011000130793074902
Iteration 217: LHS Error: 0.0010960901907097198; RHS Error 0.0010953129244540556
Iteration 218: LHS Error: 0.001091409237575891; RHS Error 0.0010906340670755496
Iteration 219: LHS Error: 0.0010867494906973696; RHS Error 0.0010859764079809596
Iteration 220: LHS Error: 0.0010821108338435192; RHS Error 0.0010813398488099058
Iteration 221: LHS Error: 0.0010774931641689967; RHS Error 0.0010767242749674284
Iteration 222: LHS Error: 0.0010728963847991824; RHS Error 0.0010721295845292555
Iteration 223: LHS Error: 0.00

Iteration 319: LHS Error: 0.0007118747355896737; RHS Error 0.0007113059608439389
Iteration 320: LHS Error: 0.0007088975501676198; RHS Error 0.0007083307223223413
Iteration 321: LHS Error: 0.0007059332396928408; RHS Error 0.0007053683550057974
Iteration 322: LHS Error: 0.0007029817430714014; RHS Error 0.0007024187984236191
Iteration 323: LHS Error: 0.0007000429995462631; RHS Error 0.0006994819922740906
Iteration 324: LHS Error: 0.0006971169772940766; RHS Error 0.0006965578756720787
Iteration 325: LHS Error: 0.000694203528037951; RHS Error 0.0006936464011428581
Iteration 326: LHS Error: 0.0006913027065809652; RHS Error 0.0006907474527502212
Iteration 327: LHS Error: 0.0006884143770640172; RHS Error 0.0006878610715305798
Iteration 328: LHS Error: 0.0006855384934633113; RHS Error 0.0006849871012228571
Iteration 329: LHS Error: 0.000682675003419092; RHS Error 0.000682125528231426
Iteration 330: LHS Error: 0.0006798238527939982; RHS Error 0.0006792762935830974
Iteration 331: LHS Error: 0.000

Iteration 426: LHS Error: 0.00045568521114004723; RHS Error 0.0004563158144069764
Iteration 427: LHS Error: 0.00045379482622100176; RHS Error 0.00045449421002473865
Iteration 428: LHS Error: 0.00045191230303974483; RHS Error 0.00045268011442063313
Iteration 429: LHS Error: 0.00045003760649681146; RHS Error 0.0004508734986196621
Iteration 430: LHS Error: 0.0004481707017325448; RHS Error 0.00044907432472273204
Iteration 431: LHS Error: 0.00044631152269646035; RHS Error 0.00044728256506976205
Iteration 432: LHS Error: 0.00044446010143556203; RHS Error 0.00044549791757809466
Iteration 433: LHS Error: 0.00044261637523750064; RHS Error 0.00044372140001940064
Iteration 434: LHS Error: 0.0004407802990064741; RHS Error 0.00044195120130553073
Iteration 435: LHS Error: 0.000438951841165075; RHS Error 0.00044018901635919233
Iteration 436: LHS Error: 0.0004371309704713777; RHS Error 0.00043843383019767646
Iteration 437: LHS Error: 0.0004353176535203951; RHS Error 0.00043668568782424637
Iteration 43

...

Iteration 1672: LHS Error: 1.8073502696322463e-06; RHS Error 0.0001174884614003292
Iteration 1673: LHS Error: 1.7990203084912448e-06; RHS Error 0.00011749664065929897
Iteration 1674: LHS Error: 1.7905281183092786e-06; RHS Error 0.00011750476989705935
Iteration 1675: LHS Error: 1.7822156857284777e-06; RHS Error 0.000117512892592553
Iteration 1676: LHS Error: 1.7738305171793867e-06; RHS Error 0.00011752094403375771
Iteration 1677: LHS Error: 1.7655241078807192e-06; RHS Error 0.00011752897152304733
Iteration 1678: LHS Error: 1.757554280428647e-06; RHS Error 0.00011753697931541664
Iteration 1679: LHS Error: 1.7490848028579864e-06; RHS Error 0.00011754490996186608
Iteration 1680: LHS Error: 1.740943676203227e-06; RHS Error 0.00011755284705769649
Iteration 1681: LHS Error: 1.7329606664708008e-06; RHS Error 0.00011756072919177473
Iteration 1682: LHS Error: 1.724651128505103e-06; RHS Error 0.00011756856075985107
Iteration 1683: LHS Error: 1.7167620886618384e-06; RHS Error 0.0001175763855528668

Iteration 1776: LHS Error: 1.109073726961174e-06; RHS Error 0.00011816553127659817
Iteration 1777: LHS Error: 1.1038545839597153e-06; RHS Error 0.00011817058920218868
Iteration 1778: LHS Error: 1.098670242959815e-06; RHS Error 0.00011817562378239888
Iteration 1779: LHS Error: 1.0935234695218232e-06; RHS Error 0.00011818063500018336
Iteration 1780: LHS Error: 1.0883735138997146e-06; RHS Error 0.00011818562308119236
Iteration 1781: LHS Error: 1.0832611865487646e-06; RHS Error 0.00011819058807328864
Iteration 1782: LHS Error: 1.0781885410350578e-06; RHS Error 0.00011819553010047369
Iteration 1783: LHS Error: 1.0731076209185599e-06; RHS Error 0.00011820044924879154
Iteration 1784: LHS Error: 1.0680663112467605e-06; RHS Error 0.0001182053456750343
Iteration 1785: LHS Error: 1.0630610355680248e-06; RHS Error 0.00011821021939374416
Iteration 1786: LHS Error: 1.058053921083272e-06; RHS Error 0.00011821507057505499
Iteration 1787: LHS Error: 1.053082657057839e-06; RHS Error 0.000118219899292774

Iteration 1881: LHS Error: 6.757502147951289e-07; RHS Error 0.00011858677978523503
Iteration 1882: LHS Error: 6.725621916094104e-07; RHS Error 0.0001185898835631867
Iteration 1883: LHS Error: 6.693890866890997e-07; RHS Error 0.0001185929727960484
Iteration 1884: LHS Error: 6.662308470865332e-07; RHS Error 0.00011859604766937312
Iteration 1885: LHS Error: 6.630873760340647e-07; RHS Error 0.00011859910843763869
Iteration 1886: LHS Error: 6.599586082557847e-07; RHS Error 0.00011860215469456335
Iteration 1887: LHS Error: 6.568444999421191e-07; RHS Error 0.00011860518681255754
Iteration 1888: LHS Error: 6.53744972779844e-07; RHS Error 0.00011860820492566593
Iteration 1889: LHS Error: 6.506599446627626e-07; RHS Error 0.00011861120890637334
Iteration 1890: LHS Error: 6.475893629099248e-07; RHS Error 0.00011861419895081889
Iteration 1891: LHS Error: 6.445331612436184e-07; RHS Error 0.00011861717504668343
Iteration 1892: LHS Error: 6.414912476094409e-07; RHS Error 0.00011862013736632995
Iterati

Iteration 1981: LHS Error: 4.2078739997475703e-07; RHS Error 0.00011883528596660226
Iteration 1982: LHS Error: 4.188422179224327e-07; RHS Error 0.00011883722911338636
Iteration 1983: LHS Error: 4.1680414959071113e-07; RHS Error 0.00011883916197700985
Iteration 1984: LHS Error: 4.1486518011224494e-07; RHS Error 0.00011884108959663714
Iteration 1985: LHS Error: 4.128671713733191e-07; RHS Error 0.00011884300433639972
Iteration 1986: LHS Error: 4.109134670278805e-07; RHS Error 0.00011884491319981514
Iteration 1987: LHS Error: 4.089820056785154e-07; RHS Error 0.00011884681110252513
Iteration 1988: LHS Error: 4.070310762596327e-07; RHS Error 0.00011884870104750169
Iteration 1989: LHS Error: 4.051040414961016e-07; RHS Error 0.0001188505818716136
Iteration 1990: LHS Error: 4.031894003976577e-07; RHS Error 0.00011885245376689127
Iteration 1991: LHS Error: 4.0127392768925935e-07; RHS Error 0.00011885431707294933
Iteration 1992: LHS Error: 3.993766160316492e-07; RHS Error 0.00011885617150465388
I

KeyboardInterrupt: 

In [ ]:
plt.plot(y_grid, π_grid[0,25])

In [ ]:
@njit(parallel=True)
def simulate_emission(e_grid, y_grid, y_start=290, z_loc=25, T=100):
    Et = np.zeros(T+1)
    for i in range(T+1):
        Et[i] = np.interp(y_start, y_grid, e_grid[z_loc])
        y_start = y_start + Et[i]
    return Et


@njit(parallel=True)
def simulate_h(e_grid, y_grid, h_grid, y_start=290, z_loc=25, T=100):
    ht = np.zeros(T+1)
    for i in range(T+1):
        e = np.interp(y_start, y_grid, e_grid[z_loc])
        ht[i] = np.interp(y_start, y_grid, h_grid[z_loc])
        y_start = y_start + e
    return ht


@njit(parallel=True)
def simulate_capital(α_k, β_k, σ_k, β_z, σ_z, ioverk, κ, Ws, k_start, N=100_000, T=100, dt=1.):
    var_z = dt*σ_z@σ_z/(1-(1-β_z*dt)**2)
    std_z = np.sqrt(var_z)

    time_periods = int(T/dt)
    z_path = np.zeros((N, time_periods+1))
    k_path = np.zeros((N, time_periods+1))
    for i in range(N):
        z0 = np.random.normal(0.,std_z)
        k0 = k_start
        z_path[i, 0] = z0
        k_path[i, 0] = k0
        for j in range(1, time_periods+1):
            k1 = k0 + (α_k + β_k*z0 + ioverk - κ/2.*ioverk**2 - σ_k@σ_k/2.)*dt + σ_k@Ws[i, j]*np.sqrt(dt)
            z1 = (1.-β_z*dt)*z0 + σ_z@Ws[i, j]*np.sqrt(dt)
            k_path[i, j] = k1
            z_path[i, j] = z1
            k0 = k1
            z0 = z1
    return k_path

In [ ]:
T = 100
Et = simulate_emission(e_grid, y_grid, y_start=290, z_loc=50, T=T)
ht = simulate_h(e_grid, y_grid, h_grid, y_start=290, z_loc=50, T=T)

In [ ]:
plt.figure()
plt.plot(Et)
plt.figure()
plt.plot(ht)

In [ ]:
T = 100
N = 1_000_000
dt = 1.
k_start = np.log(80/α)
Ws = np.random.multivariate_normal(np.zeros(2), np.eye(2), (N, int(T/dt)))
logk_path = simulate_capital(α_k, β_k, σ_k, β_z, σ_z, ioverk, κ, Ws, k_start, N=N, T=T, dt=dt)
Kt = np.percentile(np.exp(logk_path), 50, axis=0)

In [ ]:
SCCt = η*(α-ioverk)*Kt/((1.-η)*Et)
# SCCt = η*(0.025)*Kt/((1.-η)*Et)

In [ ]:
plt.plot(SCCt[:21]*1000)
plt.ylim(0, None)
plt.title('Figure 1: Social cost of carbon')
plt.xlabel('Year')
# plt.savefig('figure_1.pdf')

In [ ]:
import pickle
with open('scc_bhh.pkl', 'rb') as f:
    scc_bhh = pickle.load(f)
with open('emission.pkl', 'rb') as f:
    emission_bbh = pickle.load(f)

In [ ]:
plt.plot(scc_bhh[0]['x'], scc_bhh[0]['y'])

In [ ]:
scc_low = np.array([scc_bhh[0]['y'][4*i] for i in range(21)])

### SCC formula

$$
SCC = \frac{\eta\left[\color{red}{\alpha - \left(\frac{i}{k}\right)^*}\right]K_t^*}{(1-\eta)E_t^*}
$$

(1) in BBH, $\alpha=0.115, \left(\frac{i}{k}\right)^*=0.09$, so $\left[\alpha - \left(\frac{i}{k}\right)^*\right]=0.025$.

(2) in the new paper, $\alpha=0.2, \left(\frac{i}{k}\right)^*=0.125$, so $\left[\alpha - \left(\frac{i}{k}\right)^*\right]=0.075$.

### Adjustment cost parameters

$\alpha=0.115, \left(\frac{i}{k}\right)^*=0.09$

From FOC of $\frac{i}{k}$, we have:
$$
1-\kappa \left(\frac{i}{k}\right)^* - \frac{\delta}{\alpha-\left(\frac{i}{k}\right)^*} = 0 
$$

Therefore $\kappa=6.667$

### Log damage evolution

$$
d\log N_t = \left[\frac{d}{d\tau}\Lambda(Z_{2,t} F_t)[E_tZ_t-\rho(Z_{2,t}-\mu_2)F_t]+\frac{1}{2}\frac{d^2}{d\tau^2}\Lambda(Z_{2,t}F_t)Z_{2,t}F_t^2\sigma_2^2\right]dt + \frac{d}{d\tau} \Lambda(Z_{2,t}F_t)F_t\sqrt{Z_{2,t}}\sigma_zdW_t
$$

where

$$
dZ_t^2 = -\rho(Z_t^2 - \mu_2) dt + \sqrt{Z_t^2}\sigma_2 d W_{2,t}
$$

$$
\Lambda(\tau) = \lambda_1 \tau + \frac{\lambda_2}{2} \tau^2
$$

In [ ]:
(1-δ/(α-ioverk))/ioverk

In [ ]:
-0.0429 + ioverk - κ/2.*ioverk**2

In [ ]:
plt.plot(SCCt[:21]*1000, label='new')
plt.plot(scc_low, label='BBH low damage')
plt.ylim(0, 150)
plt.xlabel('Year')
plt.legend()
plt.title('SCC comparison')
plt.savefig('scc.pdf')

In [ ]:
plt.plot(Et[:21], label='new')
plt.plot(emission_low, label='BBH low damage')
plt.ylim(0, 20)
plt.xlabel('Year')
plt.legend()
plt.title('Emission comparison')
plt.savefig('emission.pdf')

In [ ]:
emission_low = np.array([emission_bbh[0]['y'][4*i] for i in range(21)])